# Classification primer

In [1]:
import pandas as pd
import timeit
from sklearn.linear_model import LogisticRegression

def remove_punctuation(text):
    # if text is not str:
    #     print(text)
    #     return text
    text = str(text)
    import string
    translator = str.maketrans('', '', string.punctuation)
    return text.translate(translator)

bd_load = pd.read_csv('./datasets/amazon_baby.csv')
bd = bd_load.copy()
bd.head()

,name,review,rating
0,Planetwise Flannel Wipes,"These flannel wipes are OK, but in my opinion ...",3
1,Planetwise Wipe Pouch,it came early and was not disappointed. i love...,5
2,Annas Dream Full Quilt with 2 Shams,Very soft and comfortable and warmer than it l...,5
3,Stop Pacifier Sucking without tears with Thumb...,This is a product well worth the purchase. I ...,5
4,Stop Pacifier Sucking without tears with Thumb...,All of my kids have cried non-stop when I trie...,5


## Exercise 1 (data preparation)
a) Remove punctuation from reviews using the given function.   
b) Replace all missing (nan) revies with empty "" string.  
c) Drop all the entries with rating = 3, as they have neutral sentiment.   
d) Set all positive ($\geq$4) ratings to 1 and negative($\leq$2) to -1.

In [217]:
#a)
bd['review'] = bd['review'].apply(remove_punctuation)

#short test: 
bd["review"][4] == 'All of my kids have cried nonstop when I tried to ween them off their pacifier until I found Thumbuddy To Loves Binky Fairy Puppet  It is an easy way to work with your kids to allow them to understand where their pacifier is going and help them part from itThis is a must buy book and a great gift for expecting parents  You will save them soo many headachesThanks for this book  You all rock'
remove_punctuation(bd["review"][4]) == 'All of my kids have cried nonstop when I tried to ween them off their pacifier until I found Thumbuddy To Loves Binky Fairy Puppet  It is an easy way to work with your kids to allow them to understand where their pacifier is going and help them part from itThis is a must buy book and a great gift for expecting parents  You will save them soo many headachesThanks for this book  You all rock'

True

In [218]:
#b)
bd['review'] = bd['review'].apply(lambda x: "" if x=='nan' else x)
#short test:
bd["review"][38] == bd["review"][38]

True

In [219]:
#c)
bd = bd[bd['rating'] != 3]

#short test:
sum(bd["rating"] == 3)

0

In [220]:
#d) 
bd['sentiment'] = bd['rating'].apply(lambda x: 1 if x > 3 else -1)
#short test:
print(bd[['sentiment','rating']].sample(7))
print("Test vale:")
sum(bd["sentiment"]**2 != 1)

        sentiment  rating
106543          1       5
180164          1       5
56332           1       5
24133          -1       2
183199         -1       1
106643         -1       2
65108           1       4
Test vale:


0

> 📝 <span style="color:lightblue">Komentarz:</span> Mamy więc binarną klasyfikację, na opinię pozytywnę bądź nie z pominięciem opinii neutralnych (gdzie rating$\geq$4). Bierzę się to z tąd, że nie jesteśmy w stanie jednoznacznie stwierdzić, czy opinie są bardziej negatywne, czy pozytywne. Brak takiej jednoznaczności może zaburzyć proces uczenia

## CountVectorizer
In order to analyze strings, we need to assign them numerical values. We will use one of the simplest string representation, which transforms strings into the $n$ dimensional vectors. The number of dimensions will be the size of our dictionary, and then the values of the vector will represent the number of appereances of the given word in the sentence.

In [221]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()
reviews_train_example = ["We like apples",
                   "We hate oranges",
                   "I adore bananas",
                   "We like like apples and oranges",
                   "They dislike bananas"]

X_train_example = vectorizer.fit_transform(reviews_train_example)

print(vectorizer.get_feature_names_out())
print(X_train_example.todense())



['adore' 'and' 'apples' 'bananas' 'dislike' 'hate' 'like' 'oranges' 'they'
 'we']
[[0 0 1 0 0 0 1 0 0 1]
 [0 0 0 0 0 1 0 1 0 1]
 [1 0 0 1 0 0 0 0 0 0]
 [0 1 1 0 0 0 2 1 0 1]
 [0 0 0 1 1 0 0 0 1 0]]


> 📝 <span style="color:lightblue">Komentarz:</span> Wykorzystujemy metodę 'bag of words', czyli każdy wyraz ma swoje miejsce w wektorze jako osobny index. Apples na przykład będzie zliczany na drugimindeksie a oranges na siódmym itd w każdym zdaniu.

In [222]:
reviews_test_example = ["They like bananas",
                   "We hate oranges bananas and apples",
                   "We love bananas"] #New word!

X_test_example = vectorizer.transform(reviews_test_example)
print(vectorizer.get_feature_names_out())
print(X_test_example.todense())

['adore' 'and' 'apples' 'bananas' 'dislike' 'hate' 'like' 'oranges' 'they'
 'we']
[[0 0 0 1 0 0 1 0 1 0]
 [0 1 1 1 0 1 0 1 0 1]
 [0 0 0 1 0 0 0 0 0 1]]


We should acknowledge few facts. Firstly, CountVectorizer does not take order into account. Secondly, it ignores one-letter words (this can be changed during initialization). Finally, for test values, CountVectorizer ignores words which are not in it's dictionary.

## Exercise 2 
a) Split dataset into training and test sets.     
b) Transform reviews into vectors using CountVectorizer. 

In [223]:
from sklearn.model_selection import train_test_split
#a)
bd_copy = bd.copy()
X = bd_copy['review'].values
y = bd_copy['sentiment'].values
x_train, x_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=48)

In [224]:
#b)
vectorizer = CountVectorizer()
vectorizer.fit(x_train)
x_vec_train = vectorizer.transform(x_train)
x_vec_test = vectorizer.transform(x_test)
words = vectorizer.get_feature_names_out()
print(f"Ilość słów branych pod uwagę w modelu: {len(words)}")

Ilość słów branych pod uwagę w modelu: 121305


> 📝 <span style="color:lightblue">Komentarz:</span> A więc .fit dopasowujemy nasz obiekt wektoryzujący do naszych danych, które podajemy tz. że tworzy on sobie poszczególne etykiety (w zależności od wszystkich słów w danych) dla każdego miejsca w tablicy, a następnie za pomocą transform zliczas sobie słowa i dla każdego zdania (tokenu), daje nam wektor, który jest reprezentacją tego zdania tylko, że liczbową, a więc długość wektoru to będzie ilość wszystkich unikalnych słów, we wszystkich podanych zdaniach do transform (tutaj będzie tego dużo, bardzo dużo)

## Exercise 3 
a) Train LogisticRegression model on training data (reviews processed with CountVectorizer, ratings as they were).   
b) Print 10 most positive and 10 most negative words.

In [225]:
#a)
model = LogisticRegression(solver='liblinear', max_iter=10000)
model.fit(x_vec_train, y_train)
model.predict(vectorizer.transform(['very good, im really glad i bought it']))[0]

1

> 📝 <span style="color:lightblue">Komentarz:</span> Myśle, że na komentarz tutaj zasługuje użycie innego solvera niż 'lbfgs', ponieważ mamy względnie mały dataset oraz mamy binarną klasyfikację to według dokumentacji *"For small datasets, 'liblinear' is a good choice, whereas 'sag' and 'saga' are faster for large ones;"* i rzeczywiście działą lepiej niż sag, to wyszło w trakcie testów. Widzimy, że po wytrenowaniu modelu, rzeczywiście udało się mu dobrze przewidzieć sens podanego przykładowego zdania (oczywiście jest on skrajnie pozytywny).

In [226]:
#b)
zipped = zip(model.coef_[0], vectorizer.get_feature_names_out())
zipped2 = zip(model.coef_[0], vectorizer.get_feature_names_out())
zipped_sorted_best = sorted(zipped,key=lambda x: x[0], reverse=True)
zipped_sorted_worst = sorted(zipped2,key=lambda x: x[0])
print("Najlepsze słowa:")
for coef, word in zipped_sorted_best[:11]:
    print(f"Słowo: {word:<20} Współczynnik: {coef:.4f}")

# Najgorsze współczynniki
print("\nNajgorsze słowa:")
for coef, word in zipped_sorted_worst[:11]:
    print(f"Słowo: {word:<20} Współczynnik: {coef:.4f}")

Najlepsze słowa:
Słowo: outstanding          Współczynnik: 2.1031
Słowo: saves                Współczynnik: 2.0771
Słowo: hesitate             Współczynnik: 2.0458
Słowo: lifesaver            Współczynnik: 1.9210
Słowo: rich                 Współczynnik: 1.9108
Słowo: thankful             Współczynnik: 1.8874
Słowo: con                  Współczynnik: 1.8789
Słowo: flipit               Współczynnik: 1.8568
Słowo: skeptical            Współczynnik: 1.8562
Słowo: excellent            Współczynnik: 1.8559
Słowo: penny                Współczynnik: 1.8468

Najgorsze słowa:
Słowo: dissapointed         Współczynnik: -2.7966
Słowo: disappointing        Współczynnik: -2.7691
Słowo: worst                Współczynnik: -2.7393
Słowo: unusable             Współczynnik: -2.5142
Słowo: theory               Współczynnik: -2.5067
Słowo: worthless            Współczynnik: -2.4450
Słowo: useless              Współczynnik: -2.2817
Słowo: poorly               Współczynnik: -2.2506
Słowo: unacceptable       

> 📝 <span style="color:lightblue">Komentarz:</span> Co również zasługuje na komentarz to słowa najbardziej znaczące, widzimy, że większość z nich robi sens, jednak są takie, które są wątpliwe, jak na przykład 'skeptical' albo 'con', słowa negatywne raczej mają sens 

## Exercise 4 
a) Predict the sentiment of test data reviews.   
b) Predict the sentiment of test data reviews in terms of probability.   
c) Find five most positive and most negative reviews.   
d) Calculate the accuracy of predictions.

In [227]:
#a)
test_predict = model.predict(x_vec_test)
print(f"Klasy modelu: {model.classes_}")
print(f"Wyniki przykładowe: {test_predict[0:12]}")

Klasy modelu: [-1  1]
Wyniki przykładowe: [ 1  1  1  1  1  1  1 -1  1  1  1  1]


In [228]:
#b)
test_prob_predict = model.predict_proba(x_vec_test).transpose()
test_prob_positive = test_prob_predict[1]
test_prob_negative = test_prob_predict[0]
print(f" Przykłady wyników pozytywnych: {test_prob_positive[0:12]}\nOraz wyników negatywnych {test_prob_negative[0:12]}",sep='\n')
#hint: model.predict_proba()

 Przykłady wyników pozytywnych: [0.9914131  0.99999969 0.90814348 0.59880484 0.8525614  0.84487228
 0.99560084 0.20110629 0.99805481 0.99530593 0.62988782 0.99971802]
Oraz wyników negatywnych [8.58690402e-03 3.09521254e-07 9.18565218e-02 4.01195163e-01
 1.47438595e-01 1.55127718e-01 4.39916020e-03 7.98893706e-01
 1.94518991e-03 4.69407405e-03 3.70112184e-01 2.81976015e-04]


> 📝 <span style="color:lightblue">Komentarz:</span> Oczywiście wyniki negatywne to bedzie $P_{neg}= 1-P_{poz}$.

In [229]:
#c) 
zip_opinions_positive = zip(x_test, test_prob_positive)
zip_opinions_negative = zip(x_test, test_prob_negative)

zip_pos_sorted = sorted(zip_opinions_positive,key=lambda x: x[1], reverse=True)
zip_neg_sorted = sorted(zip_opinions_negative,key=lambda x: x[1], reverse=True)

i = 1
for op,prob in zip_pos_sorted[0:5]:
    print(f"\n\n{i} with probability being positive{prob})\n{op}")
    i += 1
#hint: use the results of b)



1 with probability being positive1.0)
We are thrilled with this rear seat This little seat was a snap to install and has extended the life of our Joovy Caboose Ultralight stroller My oldest child is 4 and a half years old and very tall and he still eagerly climbs into the back of the Joovy now that we have this seatOur family has loved the Joovy Caboose Ultralight Its great and relatively easy for one parent to fold up and open At first my son loved to sit on the rear facing bench seat that comes standard in the back of the Joovy However once the novelty wore off and he grew taller he started to complain that he couldnt get comfortable as he enviously eyed his twoyearold little sister chilling in her front seat He would want to be carried instead negating the whole purpose of the double strollerThe problem is this the standard seat is small its not fixed in place it can slide forward and backward  for easy access to the underseat storage basket and theres no back  its literally a rec

In [230]:
i = 1
for op,prob in zip_neg_sorted[0:5]:
    print(f"\n\n{i} with probability being negative {prob}:\n{op}")
    i += 1



1 with probability being negative 1.0:
Please see my email to the companyHelloI am writing to voice my familys anger over your unsafe cheap cosleeper  If you recall I had a problem with my newly purchased cosleeper back in May which I immediately called about and was told to send the frame back  At that time I asked to speak to a supervisor about the situation and was told that I would be contacted shortly  However Mayra was the only one who I was able to speak with after numerous attempts to be put in contact with the supervisor  After a huge delay due to mistakes on your end I finally got the cosleeper sent back to the company after speaking with Veronica on June 13thAt this time June 13th I asked to speak with the manager of the company and Veronica told me that Sharon was not in at the time but would be in later that day  I obviously never heard from Sharon or anyone else from this company for that matter from that point on  I was inquiring to speak with the manager after voicing

> 📝 <span style="color:lightblue">Komentarz:</span> Zobaczmy więc na zdania, widzimy, że rzeczywiście wydają się być sensownie sklasyfikowane, jedne są przytłaczająco pozytywne, a drugie negatywne. A więc zadanie klasyfikacji, na pierwszy rzut oka przebiegło pomyślnie. Pragnę zaznaczyć również zirytowanie ostatniej piątej opinii negatywnej, ta osoba naprawdę była zła na ten termometr. 

In [231]:
#d) 
first_accuracy = model.score(x_vec_test, y_test)
print(first_accuracy)
time_first_model = timeit.timeit(lambda: model.predict(x_vec_test), number=3000)

0.9316062486881953


> 📝 <span style="color:lightblue">Komentarz:</span> Model mierzy dokładność na podstawie tak zadanego wzoru $Acc_{M} = \frac{ilość-poprawnie-odgadniętych}{ilość-próbek}$

## Exercise 5
In this exercise we will limit the dictionary of CountVectorizer to the set of significant words, defined below.


a) Redo exercises 2-5 using limited dictionary.   
b) Check the impact of all the words from the dictionary.   
c) Compare accuracy of predictions and the time of evaluation.

In [232]:
significant_words = ['love','great','easy','old','little','perfect','loves','well','able','car','broke','less','even','waste','disappointed','work','product','money','would','return']

x_train, x_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=50)

In [233]:
#a)
vectorizer = CountVectorizer(vocabulary=significant_words)
vectorizer.fit(x_train)
x_vec_train = vectorizer.transform(x_train)
x_vec_test = vectorizer.transform(x_test)
print(significant_words)
print(x_vec_train.todense()[:10])

['love', 'great', 'easy', 'old', 'little', 'perfect', 'loves', 'well', 'able', 'car', 'broke', 'less', 'even', 'waste', 'disappointed', 'work', 'product', 'money', 'would', 'return']
[[0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0]
 [1 1 1 0 3 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [1 1 1 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 2 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]]


In [234]:
model = LogisticRegression(max_iter=10000, solver='newton-cholesky')
model.fit(x_vec_train, y_train)

best = sorted(zip(model.coef_[0], significant_words), key=lambda x: x[0], reverse=True)
worse = sorted(zip(model.coef_[0], significant_words), key=lambda x: x[0], reverse=False)

print("Najlepsze słowa:")
for coef, word in best[:8]:
    print(f"Słowo: {word:<20} Współczynnik: {coef:.4f}")

# Najgorsze współczynniki
print("\nNajgorsze słowa:")
for coef, word in worse[:8]:
    print(f"Słowo: {word:<20} Współczynnik: {coef:.4f}")


Najlepsze słowa:
Słowo: loves                Współczynnik: 1.7044
Słowo: perfect              Współczynnik: 1.4787
Słowo: love                 Współczynnik: 1.3657
Słowo: easy                 Współczynnik: 1.1503
Słowo: great                Współczynnik: 0.9562
Słowo: little               Współczynnik: 0.4864
Słowo: well                 Współczynnik: 0.4807
Słowo: able                 Współczynnik: 0.1776

Najgorsze słowa:
Słowo: disappointed         Współczynnik: -2.3656
Słowo: return               Współczynnik: -2.0564
Słowo: waste                Współczynnik: -1.9921
Słowo: broke                Współczynnik: -1.6426
Słowo: money                Współczynnik: -0.8778
Słowo: work                 Współczynnik: -0.6464
Słowo: even                 Współczynnik: -0.5375
Słowo: would                Współczynnik: -0.3420


> 📝 <span style="color:lightblue">Komentarz:</span> Według dokumentacji: "'newton-cholesky' is a good choice for n_samples >> n_features, especially with one-hot encoded categorical features with rare categories.". Co prawda, mamy "bag of words", ale dalej spełniamy pierwszą zależność. 

In [235]:
test_predicted = model.predict(x_vec_test)
prob_good = model.predict_proba(x_vec_test).transpose()[1]

best_rev = sorted(zip(prob_good, x_test), key=lambda x: x[0], reverse=True)
worse_rev = sorted(zip(prob_good, x_test), key=lambda x: x[0], reverse=False)

i = 1
for rev in best_rev[0:5]:
    print(f"\n\n{i} with probability being positive {rev[0]})\n{rev[1]}")
    i += 1
#hint: use the results of b)



1 with probability being positive 1.0)
Check and recheck the KTan for size issues before letting your guard down and wearing NO ISSUES OCCURED however I thought I was good to go and then realized that this large was different than the large I had been given before it Odd but fabric This makes the moby wrap appear like a midevil torture devise and bonus you wont have to worry about some BabyWearing freak watching you in the parking lot at the mall as you struggle to turn a bolt of fabric into what the KTan has simplifiedseriously Dad approved and if the moby is your thing thats cool just perhaps realize that its not as easy as everyone thinks to wrap that amount of fabric around yourself evenly snug comfortable and without mummifying yourself and lord help you your baby  Another bonus to the KTAN issue with baby takes only a second to move her out of the KTAN but in a moby We thought for the first two days after returning the moby 34have you seen Katy34love it love it love it love it 

In [236]:
i = 1
for rev in worse_rev[0:5]:
    print(f"\n\n{i} with probability being positive {rev[0]})\n{rev[1]}")
    i += 1



1 with probability being positive 1.2615727050882292e-07)
This is a long review but if you read the whole thing it may save you some money and frustration If you just want to read the bit about the Bottles jump down to the 6th Paragraph and read from thereI used Avent products with my first child 5 years ago and was absolutely thrilled Back then I was a solomother on welfare so the cost was a real struggle for me but I was so thrilled with the product that I found the money to buy the things I needed for my childFive years on and oh how things have changed This time around I am married and have the funds to be able to buy whichever brand I want without money being a consideration So I eagerly stocked up on all things Avent I bought the 4 oz bottles 9 oz bottles disposable bottle kit Isis Electric Breast Pump pacifiers Bottle  Food Warmer Bottle Tote Formula Dispeser Microwave Steam Steriliser  the whole shebangThe first thing to go wrong was the Isis Breast Pump US150 from BabiesRUs 

In [237]:
second_accuracy = model.score(x_vec_test, y_test)

In [238]:
#c)
time_second_model = timeit.timeit(lambda: model.predict(x_vec_test), number=3000)

print(f"Czas dla pierwszego modelu ze wszystkimi słowami:\t {time_first_model}.\nCzas drugiego modelu z ograniczonym słownikiem:\t {time_second_model}\nCzas więc jest o {time_first_model/ time_second_model}")

print(f"Dokładność dla pierwszego {first_accuracy}.\nDokładność dla drugiego: {second_accuracy}")
#hint: %time, %timeit

Czas dla pierwszego modelu ze wszystkimi słowami:	 9.012749832996633.
Czas drugiego modelu z ograniczonym słownikiem:	 1.0506128749984782
Czas więc jest o 8.578564043401512
Dokładność dla pierwszego 0.9316062486881953.
Dokładność dla drugiego: 0.8670804473628977


> 📝 <span style="color:lightblue">Komentarz:</span> Tutaj w zasadzie powtarzamy, kroki dla bardziej zawężonego zbioru słow. Widzimy, że modelowi dalej udaje się dość dobrze rozpoznawać znaczenie zdań, co prawda, tutaj radzi sobie gorzej, bo o około 0.09. Z czego może to się brać? Oczywiste jest, że kiedy zwęziliśmy zbiór słów do analizy, ograniczyliśmy dużą ilość potencjalnie przydatnych kontekstów. Co jeszcze to fakt, że literówki pokroju "lovess" nie będą brane pod uwagę w analizie. Faktem jest, że nasz model radzi sobie troszkę gorzej, ale za to uprościliśmy znacznie jego złożoność. Zobaczmy, że już dla 3 tysięcy przewidywania jakiegoś zbioru zdań, dostajemy prawie 9 razy szybciej wynik, aniżeli w pierwszym przypadku. A więc klasycznie wracami do pytania, jakie są nasze potrzeby, czy potrzebujemy o wiele lepszą dokładność modelu, czy może ma być on mniej złożony i szybszy.